## Extended TDD Example: Monte-Carlo

### Problem: Implement and test a simple Monte-Carlo algorithm

Given an input function (energy) and starting point (density) and a temperature $T$: 

1. Compute energy at current density.
1. Move randomly chosen agent randomly left or right.
1. Compute second energy.
1. Compare the two energies:
1. If second energy is lower, accept move.
1. $\beta$ is a parameter which determines how likely
   the simulation is to move from a 'less favourable' situation to a 'more favourable' one.
1. Compute $P_0=e^{-\beta (E_1 - E_0)}$ and $P_1$ a random number between 0 and 1,
1. If $P_0 > P_1$, do the move anyway.
1. Repeat.

* the algorithm should work for (m)any energy function(s).
* there should be separate tests for separate steps! What constitutes a step?
* tests for the Monte-Carlo should not depend on other parts of code.
* Use [matplotlib](http://matplotlib.org/) to plot density at each iteration, and make an animation

## Solution

We need to break our problem down into pieces:

1. A function to generate a random change (random agent, random direction)
2. 